## 1일차 - 강화학습 예제 따라해보기
### 이 코드는 유튜버 Nicholas Renotte 님의 설명을 들으며 Google Colab PRO 환경에서 실습을 진행한 코드입니다.

- 유튜브 영상 링크 : https://youtu.be/cO5g5qLrLSo

## 실습 필요사항 설치
### Google Colab에 Tensorflow2, Keras는 설치되어있으므로 그외에 필요한 코드만 설치합니다.
#### 만약 Google Colab이 아니라면 아래의 주석처리된 코드를 주석을 해제한 후 실행하여 필요한 항목을 설치하면 됩니다.

In [ ]:
# ! pip install tensorflow==2.3.0
# ! pip install keras

In [ ]:
# install required system dependencies
!apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
!pip install gym[all]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 994 kB of archives.
After this operation, 2,981 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.9 [784 kB]
Fetched 994 kB in 0s (11.4 MB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:1

In [ ]:
!pip install keras-rl2

     |████████████████████████████████| 61kB 5.5MB/s 


## 실습 시작
### Colab 환경에서는 pyvirtualdisplay 를 통하여 Display를 생성해주어야 합니다.
- 링크 : https://bit.ly/3ylOkx9

In [ ]:
from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
import random
import gym

In [ ]:
display = Display(visible=0, size=(400, 300))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [ ]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
actions

2

In [ ]:
states

4

In [ ]:
episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode : {episode} Score : {score}")

Episode : 1 Score : 19.0
Episode : 2 Score : 24.0
Episode : 3 Score : 22.0
Episode : 4 Score : 31.0
Episode : 5 Score : 17.0
Episode : 6 Score : 41.0
Episode : 7 Score : 32.0
Episode : 8 Score : 33.0
Episode : 9 Score : 16.0
Episode : 10 Score : 12.0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))

    return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = build_model(states, actions)

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)

    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    1/10000 [..............................] - ETA: 3:27:44 - reward: 1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 84s 8ms/step - reward: 1.0000
98 episodes - episode_reward: 100.541 [8.000, 200.000] - loss: 3.341 - mae: 19.718 - mean_q: 40.010

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: 1.0000
53 episodes - episode_reward: 188.264 [145.000, 200.000] - loss: 6.328 - mae: 39.003 - mean_q: 78.806

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 84s 8ms/step - reward: 1.0000
51 episodes - episode_reward: 195.588 [160.000, 200.000] - loss: 4.443 - mae: 40.121 - mean_q: 80.876

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 86s 9ms/step - reward: 1.0000
51 episodes - episode_reward: 198.922 [171.000, 200.000] - loss: 4.850 - mae: 40.016 - mean_q: 80.532

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 85s 8ms/step - reward: 1.0000
done, took 421.829 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [ ]:
dqn.save_weights('dqn_weights.h5', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('./dqn_weights.h5')

In [ ]:
dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
